In [ ]:
import os
import subprocess
import tempfile
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI

class GitHubRepoSetup:
    def __init__(self, repo_url, target_directory=None):
        """
        Initialize GitHub repository setup
        
        Args:
            repo_url (str): URL of the GitHub repository
            target_directory (str, optional): Directory to clone repository. 
                                              Defaults to a temporary directory.
        """
        self.repo_url = repo_url
        self.target_directory = target_directory or tempfile.mkdtemp()
        
        # Initialize GitHub toolkit (optional, for additional GitHub interactions)
        github = GitHubAPIWrapper()
        self.toolkit = GitHubToolkit.from_github_api_wrapper(github)
        
        # Initialize language model
        self.llm = ChatOpenAI(temperature=0, model="gpt-4o")

    def clone_repository(self):
        """
        Clone the GitHub repository to the target directory
        """
        try:
            subprocess.run([
                'git', 'clone', self.repo_url, self.target_directory
            ], check=True)
            print(f"Repository cloned to: {self.target_directory}")
            return self.target_directory
        except subprocess.CalledProcessError as e:
            print(f"Error cloning repository: {e}")
            return None

    def detect_project_type(self):
        """
        Detect the type of project based on existing files
        """
        project_indicators = {
            'Python': ['requirements.txt', 'pyproject.toml', 'setup.py'],
            'Node.js': ['package.json'],
            'Java': ['pom.xml', 'build.gradle'],
            'Ruby': ['Gemfile'],
            'Rust': ['Cargo.toml']
        }

        for lang, files in project_indicators.items():
            for file in files:
                if os.path.exists(os.path.join(self.target_directory, file)):
                    return lang
        
        return 'Unknown'

    def setup_virtual_environment(self, project_type):
        """
        Set up a virtual environment based on project type
        """
        try:
            if project_type == 'Python':
                # Create Python virtual environment
                subprocess.run([
                    'python3', '-m', 'venv', 
                    os.path.join(self.target_directory, 'venv')
                ], check=True)
                
                # Activate virtual environment and install dependencies
                venv_python = os.path.join(self.target_directory, 'venv', 'bin', 'python')
                venv_pip = os.path.join(self.target_directory, 'venv', 'bin', 'pip')
                
                # Check for different dependency files
                if os.path.exists(os.path.join(self.target_directory, 'requirements.txt')):
                    subprocess.run([
                        venv_pip, 'install', 
                        '-r', os.path.join(self.target_directory, 'requirements.txt')
                    ], check=True)
                elif os.path.exists(os.path.join(self.target_directory, 'pyproject.toml')):
                    subprocess.run([
                        venv_pip, 'install', 'poetry'
                    ], check=True)
                    subprocess.run([
                        'poetry', 'install'
                    ], cwd=self.target_directory, check=True)
            
            elif project_type == 'Node.js':
                # Use nvm to manage Node.js version
                subprocess.run([
                    'npm', 'install'
                ], cwd=self.target_directory, check=True)
            
            print(f"Virtual environment and dependencies set up for {project_type} project")


# Simple code executor

In [4]:
! pip install langchain_experimental 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.5 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_experimental.utilities import PythonREPL
from langchain_core.messages import (
    BaseMessage, 
    HumanMessage,
    AIMessage
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from tenacity import retry, stop_after_attempt
import json

# Initialize Python REPL
repl = PythonREPL()

@tool
@retry(stop=stop_after_attempt(3))
def python_repl(
    code: str,
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.
    Always give the code in the required JSON format."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"\nSuccessfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

class CodeExecutionAgent:
    def __init__(self, base_url="http://your-api-url", api_key="your-api-key"):
        self.llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0,
            base_url=base_url,
            api_key=api_key
        )
        
        # Create the prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI assistant that can execute Python code. "
                      "Use the python_repl tool to execute code and help solve problems. "
                      "Always show your thinking process and explain what the code does."),
            MessagesPlaceholder(variable_name="messages")
        ])
        
        # Create the chain
        self.agent = self.prompt | self.llm.bind_tools([python_repl])
    
    def run(self, query: str):
        """Run a query through the agent"""
        messages = [HumanMessage(content=query)]
        response = self.agent.invoke({"messages": messages})
        return response.content

# Usage example
if __name__ == "__main__":
    agent = CodeExecutionAgent(api_key="api-key")
    
    while True:
        query = input("Enter your question (or 'exit' to quit): ")
        if query.lower() == 'exit':
            break
            
        try:
            response = agent.run(query)
            print("\nResponse:", response)
        except Exception as e:
            print(f"An error occurred: {str(e)}")

In [ ]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_experimental.utilities import PythonREPL
from langchain_core.messages import (
    BaseMessage, 
    HumanMessage,
    AIMessage
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from tenacity import retry, stop_after_attempt
import json


model = ChatOpenAI(model="gpt-4o", temperature=0, api_key="api_key")

# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal

from langchain_core.tools import tool


# Initialize Python REPL
repl = PythonREPL()

@tool
@retry(stop=stop_after_attempt(3))
def python_repl(
    code: str,
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.
    Always give the code in the required JSON format."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"\nSuccessfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

tools = [python_repl]


# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools)


In [6]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [8]:
inputs = {"messages": [("user", "Run code to calculate 8 + 6")]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Run code to calculate 8 + 6


Python REPL can execute arbitrary code. Use with caution.


================================== Ai Message ==================================
Tool Calls:
  python_repl (call_KsFlmkWkkdZfoIBgLShK181t)
 Call ID: call_KsFlmkWkkdZfoIBgLShK181t
  Args:
    code: print(8 + 6)
================================= Tool Message =================================
Name: python_repl


Successfully executed:
```python
print(8 + 6)
```
Stdout: 14


If you have completed all tasks, respond with FINAL ANSWER.
================================== Ai Message ==================================

The result of 8 + 6 is 14.
